In [2]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Define paths for dataset images and labels
data_path = "path_to_your_dataset"
images_path = os.path.join(data_path, "images")
labels_path = os.path.join(data_path, "labels")

# Define thresholds for size categorization (adjust based on dataset analysis)
SMALL_THRESHOLD = 10000  # Area below this is considered small
LARGE_THRESHOLD = 40000  # Area above this is considered large

# Custom dataset class for loading chicken images and annotations
class ChickenDataset(Dataset):
    def __init__(self, images_path, labels_path, transform=None):
        self.images_path = images_path
        self.labels_path = labels_path
        self.image_files = [f for f in os.listdir(images_path) if f.endswith(".jpg")]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_path, img_name)
        label_path = os.path.join(self.labels_path, img_name.replace(".jpg", ".txt"))

        # Load and preprocess image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = cv2.resize(image, (224, 224))  # Resize to match CNN input size

        # Load YOLOv8 annotation file
        with open(label_path, "r") as f:
            lines = f.readlines()
            if not lines:
                raise ValueError(f"No annotations found for {img_name}")
            _, x, y, w, h = map(float, lines[0].strip().split())

        # Convert YOLO format (relative coordinates) to absolute pixel values
        img_height, img_width, _ = image.shape
        w, h = int(w * img_width), int(h * img_height)
        bbox_area = w * h  # Compute bounding box area

        # Assign size category based on bounding box area
        if bbox_area < SMALL_THRESHOLD:
            label = 0  # Small
        elif bbox_area > LARGE_THRESHOLD:
            label = 2  # Large
        else:
            label = 1  # Medium

        # Apply transformations if specified
        if self.transform:
            image = self.transform(image)

        return image, label

# Define data augmentation and preprocessing transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(15),  # Random rotation to improve model generalization
    transforms.ToTensor(),  # Convert to tensor format for PyTorch
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values
])

# Load dataset and create a DataLoader for batching
dataset = ChickenDataset(images_path, labels_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)  # Shuffle for better training

# Define CNN model (ResNet18 with modified classification layer)
class ChickenSizeClassifier(nn.Module):
    def __init__(self):
        super(ChickenSizeClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)  # Load pre-trained ResNet18
        self.model.fc = nn.Linear(512, 3)  # Modify final layer for 3-class classification (Small, Medium, Large)
    
    def forward(self, x):
        return self.model(x)

# Function to train the model
def train_model():
    model = ChickenSizeClassifier()
    criterion = nn.CrossEntropyLoss()  # Loss function for classification
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001
    model.train()  # Set model to training mode
    
    for epoch in range(10):  # Train for 10 epochs
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.float(), labels  # Convert images to float tensors
            optimizer.zero_grad()  # Reset gradients before backpropagation
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagate gradients
            optimizer.step()  # Update model weights
            running_loss += loss.item()
        
        # Print loss after each epoch to track training progress
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}")
    
    torch.save(model.state_dict(), "chicken_size_model.pth")  # Save trained model
    print("Model training complete!")

# Execute training function
train_model()
